In [227]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt


def remove_digits_parentheses(text):
    pattern = r'\([^)]*\)|\d+'
    return re.sub(pattern, '', text)

def process_energy_data(file_path, sheet_name, start_row, country_mapping, stop_country, max_rows=20):
    new_column_names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    data = pd.read_excel(file_path, sheet_name=sheet_name, header=start_row, usecols='C:F')
    data.columns = new_column_names
    data = data.replace('...', np.NaN)

    data['Country'] = data['Country'].astype(str)
    data['Country'] = data['Country'].apply(remove_digits_parentheses)
    data['Country'] = data['Country'].replace(country_mapping)
    data['Country'].fillna('', inplace=True)

    data['Energy Supply'] = data['Energy Supply'] * 1000000

    return data
#gdp
def process_gpd_file(file_path):
    gpd_data = pd.read_csv(file_path, skiprows=4)
    return gpd_data
#magojr
def process_scimagojr_file(file_path):
    scimago_data = pd.read_excel(file_path, sheet_name='Sheet1', header=0, usecols="A:H", nrows=191, engine='openpyxl')
    return scimago_data

# file
file_path_energy = r'D:\Навчання\ХПІ\Pyton\L3\En_In.xls'
file_path_gpd = r'D:\Навчання\ХПІ\Pyton\L3\gpd.csv'
file_path_scimagojr = r'D:\Навчання\ХПІ\Pyton\L3\scimagojr.xlsx'

# energy
sheet_name_energy = 'Energy'
start_row_energy = 17 
country_mapping_energy = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
stop_country_energy = 'Zimbabwe'
#unite
# Функція обробки даних з врахуванням NaN у відсутніх комірках
def merge_and_process_data(file_path_energy, file_path_gpd, file_path_scimagojr):
    energy_data = process_energy_data(file_path_energy, sheet_name_energy, start_row_energy, country_mapping_energy, stop_country_energy)
    gpd_data = process_gpd_file(file_path_gpd)
    scimagojr_data = process_scimagojr_file(file_path_scimagojr)
    
    merged_data = pd.merge(scimagojr_data, energy_data, how='left', left_on='Country', right_on='Country')
    merged_data = pd.merge(merged_data, gpd_data, how='left', left_on='Country', right_on='Country Name')
    
    years = [str(year) for year in range(2006, 2016)]
    selected_columns = ['Country', 'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
                        'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] + years
    selected_data = merged_data.nsmallest(15, 'Rank')[selected_columns]
    selected_data.set_index('Country', inplace=True)

    return selected_data



# Задача 8
def avg_gdp_last_10_years(df):
    # Отримання стовпців з роками
    year_columns = df.columns[-10:]
    # Обчислення середнього ВВП за останні 10 років
    df['Avg GDP Last 10 Years'] = df[year_columns].mean(axis=1)
    return df['Avg GDP Last 10 Years']

# Задача 9
def change_gdp_for_5th_country(df):
    avg_gdp = avg_gdp_last_10_years(df)
    fifth_country = avg_gdp.nlargest(5).iloc[-1]  
    country_name = avg_gdp[avg_gdp == fifth_country].index[0]  
    gdp_change = df.loc[country_name][-1] - df.loc[country_name][-11]  
    return country_name, gdp_change

# Задача 10
def max_renewable_country(df):
    max_renewable = df['% Renewable'].idxmax()  # Назва країни з максимальним % Renewable
    max_renewable_value = df.loc[max_renewable]['% Renewable']  # Значення максимального % Renewable
    return max_renewable, max_renewable_value

# Задача 11
def sixth_population_country(df):
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    sixth_pop_country = df['Population'].nlargest(6).iloc[-1]
    country_name = df[df['Population'] == sixth_pop_country].index[0]
    return country_name

# Задача 12
def correlation_citations_energy(df):
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    df['Citations per Capita'] = df['Citable documents'] / df['Population']
    correlation = df['Citations per Capita'].corr(df['Energy Supply per Capita'])
    return correlation

# Задача 13
def compare_renewable_to_median(df):
    median_renewable = df['% Renewable'].median()
    df['Renewable Above Median'] = df['% Renewable'].apply(lambda x: 1 if x >= median_renewable else 0)
    return df['Renewable Above Median']

# Задача 14
ContinentDict = {'China':'Asia', 'United States':'North America', 'Japan':'Asia', 'United Kingdom':'Europe', 'Russian Federation':'Europe', 'Canada':'North America', 'Germany':'Europe', 'India':'Asia', 'France':'Europe', 'South Korea':'Asia', 'Italy':'Europe', 'Spain':'Europe', 'Iran':'Asia', 'Australia':'Australia', 'Brazil':'South America'}

def group_by_continent(df, continent_dict):
    df['Continent'] = df.index.map(continent_dict)  # Додавання стовпця з континентами
    population = df['Energy Supply'] / df['Energy Supply per Capita']
    grouped = population.groupby(df['Continent'])  # Групування за континентами
    summary_df = pd.DataFrame({'size': grouped.size(),  # Розмір вибірки
                               'sum': grouped.sum(),  # Сума населення
                               'mean': grouped.mean(),  # Середнє населення
                               'std': grouped.std()})  # Стандартне відхилення
    return summary_df





formatted_df = merge_and_process_data(file_path_energy, file_path_gpd, file_path_scimagojr)
formatted_df






Energy Data:
          Country  Energy Supply  Energy Supply per Capita  % Renewable
0     Afghanistan   3.210000e+08                      10.0    78.669280
1         Albania   1.020000e+08                      35.0   100.000000
2         Algeria   1.959000e+09                      51.0     0.551010
3  American Samoa            NaN                       NaN     0.641026
4         Andorra   9.000000e+06                     121.0    88.695650

GDP Data:
  Country Name Country Code                            Indicator Name  \
0        Aruba          ABW  GDP at market prices (constant 2010 US$)   
1      Andorra          AND  GDP at market prices (constant 2010 US$)   
2  Afghanistan          AFG  GDP at market prices (constant 2010 US$)   
3       Angola          AGO  GDP at market prices (constant 2010 US$)   
4      Albania          ALB  GDP at market prices (constant 2010 US$)   

   Indicator Code  1960  1961  1962  1963  1964  1965  ...          2006  \
0  NY.GDP.MKTP.KD   NaN   NaN

In [236]:
grouped_continent_data = group_by_continent(formatted_df, ContinentDict)
print("Grouped by Continent:")
print(grouped_continent_data)

Grouped by Continent:
               size           sum          mean           std
Continent                                                    
Asia              5  2.821591e+09  7.053977e+08  7.138779e+08
Australia         1  2.331602e+07  2.331602e+07           NaN
Europe            6  4.579297e+08  7.632161e+07  3.464767e+07
North America     2  3.528552e+08  1.764276e+08  1.996696e+08
South America     1  2.059153e+08  2.059153e+08           NaN


In [235]:
compare_renewable_to_median(formatted_df)

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: Renewable Above Median, dtype: int64

In [234]:
correlation_citations_energy(formatted_df)

0.7901935488300874

In [233]:
sixth_population_country(formatted_df)

'Japan'

In [232]:
max_renewable_country(formatted_df)

('Brazil', 69.64803)

In [230]:
change_gdp_for_5th_country(formatted_df)

('France', 82012239735.09668)

In [231]:
avg_gdp_last_10_years(formatted_df)

Country
China                 6.610156e+12
United States         1.542784e+13
Japan                 5.547277e+12
United Kingdom        2.495485e+12
Russian Federation    1.585403e+12
Canada                1.671323e+12
Germany               3.510800e+12
India                 1.825175e+12
France                2.689926e+12
South Korea                    NaN
Italy                 2.111074e+12
Spain                 1.418440e+12
Iran                           NaN
Australia             1.179816e+12
Brazil                2.228057e+12
Name: Avg GDP Last 10 Years, dtype: float64